In [1]:
%matplotlib inline
#! switch_R 3.3
%load_ext rpy2.ipython

In [2]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import statsmodels.stats.api as sms
from pyspan import cross_validation as cv
from pyspan.ratings_task.analysis import *
from pyspan.config import *
from pyspan.valence import *
assert settings["mode"] == "crec"
INPUT_DIR = paths["input_dir"]
METRICS_DIR = paths["metrics_dir"]

/Users/sabinasloman/git/talking-politics/pyspan/config.py:19: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_i

# Participants

In [3]:
# n
print len(minidf)
# Mean age and standard error of the mean
print np.mean(minidf.age[~np.isnan(minidf.age)]), stats.sem(minidf.age[~np.isnan(minidf.age)])
# n identifying as male/female
print len(minidf.loc[minidf.gender == "M"]), len(minidf.loc[minidf.gender == "F"])
# % voted
print np.mean(minidf.voted)

147
37.65986394557823 0.8431946541195345
74 72
0.8163265306122449


# Results

In [4]:
dat = minidf[map(str, p_ixs)].values
ddat = dat[:,partisan.loc[p_ixs].PKL_D > 0]
rdat = dat[:,partisan.loc[p_ixs].PKL_D < 0]

In [5]:
rmu = np.mean(rdat[~np.isnan(rdat)])
dmu = np.mean(ddat[~np.isnan(ddat)])

Mean participant rating on the Republican and Democratic items (add 1 to obtain the numbers reported in the paper, as those are described on a 1--6 scale, while these are on a 0--5 scale).

In [6]:
rmu, dmu

(2.8064862927790717, 2.2983811626195734)

Perform one- and two-sample $t$-tests, using the clustered standard errors function provided by Arai (2011).

In [7]:
%%R
source("../clmclx.R")

One-sample test (judgments of the Republican words).

In [8]:
%%R -i rdat
y <- c(t(rdat)) - 2.5
nr <- dim(rdat)[1]
kr <- dim(rdat)[2]
idxn <- rep(1:nr, each = kr)
idxn <- idxn[which(!is.na(y))]
idxk <- rep(1:kr, nr)
idxk <- idxk[which(!is.na(y))]
fit <- lm(y ~ 1)
print(fit$df.residual)
fit <- lm(y ~ 1)
res <- mclx(fit, 1, idxn, idxk)
se <- res[1,2]
t <- res[1,3]
p <- res[1,4] / 2
print(paste(se, t, p))

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: zoo

  warnings.warn(x, RRuntimeWarning)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘zoo’


  warnings.warn(x, RRuntimeWarning)
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


  warnings.warn(x, RRuntimeWarning)


[1] 5580
[1] "0.0812042442925946 3.77426445438914 8.10728057510572e-05"


One-sample test (judgments of the Democratic words).

In [9]:
%%R -i ddat
y <- c(t(ddat)) - 2.5
nd <- dim(ddat)[1]
kd <- dim(ddat)[2]
idxn <- rep(1:nd, each = kd)
idxn <- idxn[which(!is.na(y))]
idxk <- rep(1:kd, nd)
idxk <- idxk[which(!is.na(y))]
fit <- lm(y ~ 1)
print(fit$df.residual)
res <- mclx(fit, 1, idxn, idxk)
se <- res[1,2]
t <- res[1,3]
p <- res[1,4] / 2
print(paste(se, t, p))

[1] 5435
[1] "0.121547860301295 -1.65876089369777 0.048610842215481"


Two-sample test (testing the difference in means).

In [10]:
%%R
y <- c(t(ddat), t(rdat))
idvs <- c(rep(1:nd, each = kd), rep((nd+1):(nd+nr), each = kr))
idxn <- idvs[which(!is.na(y))]
items <- c(rep(1:kd, nd), rep((kd+1):(kd+kr), nr))
idxk <- items[which(!is.na(y))]
groups <- c(rep(0, nd*kd), rep(1, nr*kr))
fit <- lm(y ~ groups)
print(fit$df.residual)
res <- mclx(fit, 1, idxn, idxk)
se <- res[2,2]
t <- res[2,3]
p <- res[2,4] / 2
print(paste(se, t, p))

[1] 11015
[1] "0.14523446030682 3.49851632378421 0.00023485058296235"


## Effect size

Using the formula from [Rouder et al. (2012)](http://pcl.missouri.edu/sites/default/files/Rouder.JMP_.2012.pdf) and explained in [Jake Westfall's blog post](http://jakewestfall.org/blog/index.php/2016/03/25/five-different-cohens-d-statistics-for-within-subject-designs/).

In [11]:
%%R 
library("lme4")
mod <- lmer(y ~ groups + (groups|idvs) + (1|items))
delta <- summary(mod)$coefficients[2,1]
sig <- sigma(mod)
delta / sig

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: Matrix

  warnings.warn(x, RRuntimeWarning)


[1] 0.4063363


## Item-level analyses

In [12]:
def calc_accuracy(i):
    polarity = signal_df.loc[partisan.loc[i]["word"]].rmetric > 0
    judgments = minidf[str(i)]
    judgments = judgments[~np.isnan(judgments)]
    return len(judgments[judgments > 2.5 if polarity else judgments < 2.5]), len(judgments)
v_calc_accuracy = np.vectorize(calc_accuracy)

In [13]:
p, n = v_calc_accuracy(p_ixs)
p = p / n

Number of words correctly classified.

In [14]:
dixs = [ i for i in range(len(p_ixs)) if signal_df.loc[partisan.loc[p_ixs[i]]["word"]].dmetric > 0 ]
rixs = [ i for i in range(len(p_ixs)) if signal_df.loc[partisan.loc[p_ixs[i]]["word"]].rmetric > 0 ]

In [15]:
(p[dixs] > .5).sum(), (p[rixs] > .5).sum(), (p > .5).sum()

(24, 26, 50)

Average item-level accuracy.

In [16]:
np.mean(p), stats.sem(p)

(0.5861659366943126, 0.023279929676539295)

In [17]:
t, p_ = stats.ttest_1samp(p, .5)
t, p_ / 2, len(p)-1 

(3.701297121234333, 0.00020504711762496346, 74)

Mann-Whitney U test

For each word, compute the mean of the judgments across subjects, and perform a Mann-Whitney U test to determine whether the Republican words are as likely to have a higher rank than Democratic words as to have a lower rank.

In [18]:
dmeans = np.mean(ddat, axis=0)
rmeans = np.mean(rdat, axis=0)

In [19]:
stats.mannwhitneyu(dmeans, rmeans, alternative="less")

MannwhitneyuResult(statistic=416.0, pvalue=0.001197878011080101)

### Figure

This creates a csv file called `figure.csv` with columns `word`, `dmetric`, `rmetric`, `ratings_mean`, `ratings_std` and `frac_r`. The script `figures/figures_2_4.py` creates Fig 2.

In [20]:
perceps = PerceptualData(ixs = p_ixs)
perceps.get_discriminability_by_word()
perceps.item_level_res(signal_df, savetofile="figure.csv")

## Participant-level analyses

Accuracies

In [21]:
signals = np.array(signals)
prop_right = []
for i in minidf.index:
    perceps = minidf.loc[i][map(str, p_ixs)].values
    perceps = np.array(list(map(float, perceps)))
    nright = len(perceps[((perceps < 2.5) & (signals < 0)) | ((perceps > 2.5) & (signals > 0))])
    prop_right.append(nright / len(perceps[~np.isnan(perceps)]))

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  
/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  


In [22]:
np.mean(prop_right), stats.sem(prop_right)

(0.5861519836862302, 0.005877149218751656)

In [23]:
t, p = stats.ttest_1samp(prop_right, .5)
t, p / 2, len(prop_right) - 1, \
    len([ prop for prop in prop_right if prop > .5 ])

(14.658804886449602, 8.686107635321322e-31, 146, 127)

Accuracies on the Democratic items...

In [24]:
prop_right = []
for i in minidf.index:
    perceps = minidf.loc[i][map(str, p_ixs)].values
    perceps = np.array(list(map(float, perceps)))
    perceps = perceps[signals < 0]
    nright = len(perceps[perceps < 2.5])
    prop_right.append(nright / len(perceps[~np.isnan(perceps)]))
np.mean(prop_right), stats.sem(prop_right)

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  


(0.5751776557899008, 0.010990721007601595)

... and on the Republican items.

In [25]:
prop_right = []
for i in minidf.index:
    perceps = minidf.loc[i][map(str, p_ixs)].values
    perceps = np.array(list(map(float, perceps)))
    perceps = perceps[signals > 0]
    nright = len(perceps[perceps > 2.5])
    prop_right.append(nright / len(perceps[~np.isnan(perceps)]))
np.mean(prop_right), stats.sem(prop_right)

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  


(0.5968637810743074, 0.009098182686866388)

Discriminabilities

In [26]:
signals = np.array(signals)
prop_right = []
for i in minidf.index:
    perceps = minidf.loc[i][map(str, p_ixs)].values
    perceps = np.array(list(map(float, perceps)))
    rjudgments = perceps[signals > 0]
    rjudgments = rjudgments[~np.isnan(rjudgments)]
    djudgments = perceps[signals < 0]
    djudgments = djudgments[~np.isnan(djudgments)]
    prop_right.append(cohensd(rjudgments, djudgments))

In [27]:
np.mean(prop_right), stats.sem(prop_right)

(0.4033623843300961, 0.02592686178162114)

In [28]:
t, p = stats.ttest_1samp(prop_right, 0)
t, p / 2, len(prop_right) - 1

(15.557701804698516, 4.249474340581165e-33, 146)

# Appendix E

Testing for individual-level differences in discriminability between Democrats and Republicans

In [29]:
prop_right_r = []
for i in minidf.loc[minidf.party == "Republican"].index:
    perceps = minidf.loc[i][map(str, p_ixs)].values
    perceps = np.array(list(map(float, perceps)))
    rjudgments = perceps[signals > 0]
    rjudgments = rjudgments[~np.isnan(rjudgments)]
    djudgments = perceps[signals < 0]
    djudgments = djudgments[~np.isnan(djudgments)]
    prop_right_r.append(cohensd(rjudgments, djudgments))
    
prop_right_d = []
for i in minidf.loc[minidf.party == "Democrat"].index:
    perceps = minidf.loc[i][map(str, p_ixs)].values
    perceps = np.array(list(map(float, perceps)))
    rjudgments = perceps[signals > 0]
    rjudgments = rjudgments[~np.isnan(rjudgments)]
    djudgments = perceps[signals < 0]
    djudgments = djudgments[~np.isnan(djudgments)]
    prop_right_d.append(cohensd(rjudgments, djudgments))
    
prop_right_r = np.array(prop_right_r)
prop_right_d = np.array(prop_right_d)

In [30]:
np.mean(prop_right_r), stats.sem(prop_right_r), \
    np.mean(prop_right_d), stats.sem(prop_right_d)

(0.24318006794665611,
 0.05637666271252923,
 0.4697651543599723,
 0.035351293795834404)

In [31]:
cm = sms.CompareMeans(sms.DescrStatsW(prop_right_d), sms.DescrStatsW(prop_right_r))
cm.ttest_ind(usevar="unequal")

(3.4050653585985096, 0.0011332554083023421, 65.56602803605274)

Logistic regression further investigating individual differences

Correct ~ logodds(word) + log(P(word)) + party(participant) + valence(word) + party(participant) $\times$ valence(word) $\times$ is\_republican(word) + party\_identity + party\_identity $\times$ party(participant) + political\_engagement + political\_engagement $\times$ party(participant) + $\ldots{}$

In [32]:
lr_data = minidf.copy()
lr_data = lr_data.loc[lr_data.party.isin([ "Democrat", "Republican" ])]
n = len(lr_data)
items = partisan.loc[p_ixs].word
n_vars = 9

# Add dummy columns
for i in range(1, n):
    ids = np.zeros(n)
    ids[i] = 1
    lr_data["participant{}".format(i)] = ids
    
Y_full = np.ravel(lr_data[map(str, p_ixs)])
Y = Y_full > 2.5
X = np.full((n * len(p_ixs), n_vars + n - 1 + len(p_ixs) - 1), np.nan)
# The log odds that the word was said by a Republican
vf = np.vectorize(lambda word: abs(signal_df.loc[word]["rmetric"]))
X[:,0] = np.tile(vf(items), n)
# The log probability of hearing the word
vf = np.vectorize(lambda word: math.log(sum(freq_df.loc[(word,["dmetric","rmetric"])].values) / n_utterances, 2))
X[:,1] = np.tile(vf(items), n)
# Participant's political identity
vf = np.vectorize(lambda pid: 1 if pid == "Republican" else -1 if pid == "Democrat" else 0)
pids = np.repeat(vf(lr_data.party), len(p_ixs))
X[:,2] = pids
# Valence of word
vf = np.vectorize(lambda word: get_valence(word)[0]-5)
X[:,3] = np.tile(vf(items), n)
# Valence of word * Participant's party identity * Word is Republican
vf = np.vectorize(lambda word: signal_df.loc[word]["rmetric"])
v = np.tile(vf(items), n)
X[:,4] = pids * X[:,3] * np.sign(v) 
# Party identity
X[:,5] = np.repeat(lr_data.party_identity, len(p_ixs))
# Party affiliation x party identity
X[:,6] = pids * X[:,5]
# Political engagement
X[:,7] = np.repeat(lr_data.political_engagement, len(p_ixs))
# Party affiliation x political engagement
X[:,8] = pids * X[:,7]

# Participant dummies
for i in range(n_vars, n_vars + n - 1):
    X[:,i] = np.repeat(lr_data["participant{}".format(i-n_vars+1)], len(p_ixs))

# Item dummies
for i in range(1,len(p_ixs)):
    ix = n_vars+n-1+i-1
    ind_vec = np.zeros(len(p_ixs))
    ind_vec[i] = 1
    X[:,ix] = np.tile(ind_vec, n)

vf = np.vectorize(lambda word: signal_df.loc[word]["rmetric"] > 0)
polarity = vf(items)
polarity = np.tile(polarity, n)
Y = (Y & polarity) | (~Y & ~polarity)

Y = Y[~np.isnan(Y_full)]
X = X[~np.isnan(Y_full)]
Y_full = Y_full[~np.isnan(Y_full)]
X = X[~np.isnan(Y)]
Y_full = Y_full[~np.isnan(Y)]
Y = Y[~np.isnan(Y)]
Y = Y[~np.isnan(X).any(axis = 1)]
Y_full = Y_full[~np.isnan(X).any(axis = 1)]
X = X[~np.isnan(X).any(axis = 1),:]

# Since we don't have valence data for some of the words, their observations will be deleted and their indicator columns will
# just be clutter (and cause problems with model estimation)
colixs = np.arange(0,X.shape[1])
to_del = np.where(np.apply_along_axis(lambda x: np.all(x == 0), 0, X[:,n_vars+n-1:-1]))[0] + n_vars+n-1
colixs = [ colix for colix in colixs if colix not in to_del ]
X = X[:,colixs]

X = stats.mstats.zscore(X)

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


In [33]:
X.shape, np.linalg.matrix_rank(X)

((6726, 174), 166)

In [34]:
X[:,:n_vars].shape, np.linalg.matrix_rank(X[:,:n_vars])

((6726, 9), 9)

The matrix with fixed effects is not full rank, but the matrix without fixed effects is, so estimate the model without fixed effects.

In [35]:
logit = SparseLR(Y, X[:,:n_vars]); logit.coef

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/politics-test-bed/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([ 0.32761308,  0.13406804, -0.08107084,  0.19290955,  0.30146516,
        0.02403751, -0.01787496,  0.06184514,  0.00804689])

In [36]:
logit.n, logit.auc

(6726, 0.6130553253028238)

How many items are effectively excluded on the basis of missing valence data?

In [37]:
vf = np.vectorize(lambda word: get_valence(word)[0]-5)
v = vf(items)
len(v[np.isnan(v)])

7